In [14]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import svm

from sklearn import tree
from sklearn import neighbors

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic


from sklearn.model_selection import cross_val_score, KFold

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn import linear_model
from sklearn import ensemble 
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.neighbors import KNeighborsRegressor


from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

#import xgboost as xgb

#from lightgbm import LGBMRegressor
#from autosklearn.regression import AutoSklearnRegressor
#from autosklearn.metrics import mean_absolute_error as auto_mean_absolute_error

#from numba import njit
#import shap


In [2]:
cwd = os.getcwd()

# Read and Scale the Data

In [4]:
train_data = pd.read_excel (r'data.xlsx') 

name=['Z','N','A']  #0.940609 1.02737  1.14384

x_train = pd.DataFrame(train_data,  columns=name)  
y_train = pd.DataFrame(train_data, columns= ['MeV'])

test_size = 0.70  # You can adjust this to set the proportion of the test set
random_state = 100  # You can choose any random_state value for reproducibility

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_size, random_state=random_state)

train_len=len(x_train)
test_len=len(x_test)

#test_data= pd.read_excel (r'test_fixed.xls')
#x_test = pd.DataFrame(test_data,  columns=name)  
#y_test = pd.DataFrame(test_data, columns= ['MeV'])

#test_len=len(x_test)

extra_data= pd.read_excel (r'NEW_EXP_DATA_FOR_EXTRAPOLATION.xlsx')
x_extra = pd.DataFrame(extra_data,  columns=name)  
y_extra = pd.DataFrame(extra_data, columns= ['MeV'])

inp=np.concatenate((x_train,x_test,x_extra),axis=0)
out=np.concatenate((y_train,y_test,y_extra),axis=0)
train_test_len=train_len+test_len

##################################
## scale
##################################
# Initialize the StandardScaler for input data
sc_X = StandardScaler()

# Fit the scaler on the training data
sc_X.fit(x_train)

# Transform the training and testing data using the fitted scaler
x_train = sc_X.transform(x_train)
x_test = sc_X.transform(x_test)
x_extra = sc_X.transform(x_extra)

# Initialize the StandardScaler for output data
sc_Y = StandardScaler()

# Fit the scaler on the training data
sc_Y.fit(y_train)

# Transform the training and testing data using the fitted scaler
y_train = sc_Y.transform(y_train)
y_test = sc_Y.transform(y_test)
y_extra = sc_Y.transform(y_extra)

# For input data
print("Input Data:")
print("Mean:", sc_X.mean_)
print("Standard Deviation:", sc_X.scale_)

# For output data
print("\nOutput Data:")
print("Mean:", sc_Y.mean_)
print("Standard Deviation:", sc_Y.scale_)
################################
name+=['original','prediction']


Input Data:
Mean: [ 56.71328671  79.04755245 135.76083916]
Standard Deviation: [24.70939415 38.28400282 62.61852653]

Output Data:
Mean: [-40.76148206]
Standard Deviation: [42.44917003]


In [5]:
print(x_train)

[[ 0.05207385  0.20772247  0.14754676]
 [-0.31216009 -0.60201522 -0.49124182]
 [ 0.98289392  1.30478644  1.18557821]
 ...
 [-1.68815498 -1.54235576 -1.60912185]
 [-0.75733491 -0.62813579 -0.6828784 ]
 [ 1.34712786  1.72271557  1.58482108]]


In [15]:
print(x_test)

[[ 0.9176803   0.73523509  0.81212322]
 [-0.30999235 -0.00573276 -0.12603237]
 [ 0.83583546  0.33828803  0.53714658]
 ...
 [ 1.16321483  1.18510843  1.18415044]
 [ 0.67214577  0.52352999  0.58567187]
 [ 0.71306819  0.39121431  0.52097149]]


In [16]:
print(x_extra)

[[-2.02873405 -1.91107865 -1.96999337]
 [-2.02873405 -1.88461551 -1.95381827]
 [-2.02873405 -1.85815238 -1.93764318]
 ...
 [ 2.02258568  1.95253942  1.99290526]
 [ 2.02258568  1.97900255  2.00908036]
 [ 2.10443052  2.03192883  2.07378075]]


# Linear Models

## 3) [Gaussian Process](https://scikit-learn.org/stable/auto_examples/gaussian_process/plot_gpr_noisy_targets.html#sphx-glr-auto-examples-gaussian-process-plot-gpr-noisy-targets-py)
### parameters:
### Kernel selected as RBF + White

In [8]:
os.chdir(cwd)
analyze_name="GP_Regression"
if not (os.path.exists("./"+str(analyze_name))):
    os.makedirs("./"+str(analyze_name));
os.chdir("./"+str(analyze_name));
wfile=open("results_"+str(analyze_name)+"_selected.txt","a")
Scale=[1.0] 
Alpha=[1e-09] 
Normalize=[False] 

# K-fold cross-validation
#kf = KFold(n_splits=5, shuffle=True, random_state=42)

for s in Scale:
    for a in Alpha:
        for n in Normalize:
            wfile.write(analyze_name+"\n")
            wfile.write("Scale : {} Alpha: {} Normalize: {} \n".format(str(s),str(a),str(n)))
            wfile.write("----------------------------------\n")
            ###############################
            ## Model
            ################################
            #kernel = 1.0 * Matern(length_scale=1.0, nu=2.0,length_scale_bounds=(1e-10, 1e6))+\
            #WhiteKernel(noise_level=1, noise_level_bounds=(1e-20, 1e4))
            
            kernel = 1.0 * RBF(length_scale=s, length_scale_bounds=(1e-4, 1e5)) +\
            WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e1))
            gpr = GaussianProcessRegressor(kernel=kernel,alpha=a,normalize_y=n).fit(x_train, y_train)

        
            ######## EY: The score is printed  ########  31/10/2023 
            score = gpr.score(x_train, y_train)
             # Print the score
            print("R^2 Score:", score)
            ###########################################  

            # Cross-validation
            #cv_scores = cross_val_score(gpr, x_train, y_train.flatten(), cv=kf, scoring='neg_mean_squared_error')
            #mean_cv_score = np.mean(cv_scores)
            #wfile.write(f'Mean Cross-Validation Score: {mean_cv_score}\n')

            # Fit the GPR model on the entire training set
            #gpr.fit(x_train, y_train.flatten())
            
            
            pred_train=gpr.predict(x_train)  #EY: Train data added
            pred_train=pred_train.reshape(-1,1)  
                
            pred=gpr.predict(x_test)
            pred=pred.reshape(-1,1)
            
            pred_extra=gpr.predict(x_extra)
            pred_extra=pred_extra.reshape(-1,1)

            ########### EY: Standart deviations are calculated  ################### 31/10/2023
            pred_train_mean, pred_train_std = gpr.predict(x_train, return_std=True)     # Training set 
            pred_train_mean = pred_train_mean.reshape(-1, 1)  
            pred_train_std = pred_train_std.reshape(-1, 1)
            
            pred_mean, pred_std = gpr.predict(x_test, return_std=True)               # Test set
            pred_mean = pred_mean.reshape(-1, 1)
            pred_std = pred_std.reshape(-1, 1)
            
            pred_extra_mean, pred_extra_std = gpr.predict(x_extra, return_std=True)       # Extrapolation set 
            pred_extra_mean = pred_extra_mean.reshape(-1, 1)
            pred_extra_std = pred_extra_std.reshape(-1, 1)

            # Scale and normalize the standard deviations
            #sc_std = StandardScaler()
            #sc_std.fit(pred_train_std)  # Fit the scaler to the training standard deviations
            #pred_train_std_normalized = sc_std.transform(pred_train_std)
            #pred_std_normalized = sc_std.transform(pred_std)
            #pred_extra_std_normalized = sc_std.transform(pred_extra_std)
            #######################################################################  
            
            ################################
            ## Train data predictions
            ################################
            data=np.concatenate((sc_X.inverse_transform(x_train),sc_Y.inverse_transform(y_train),sc_Y.inverse_transform(pred_train)),axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('Train_data_predictions.xlsx', index=False, header=name)
    
            ################################
            ## Test data predictions
            ################################
            data=np.concatenate((sc_X.inverse_transform(x_test),sc_Y.inverse_transform(y_test),sc_Y.inverse_transform(pred)),axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('predictions.xlsx', index=False, header=name)
            ################################
            ## Extrapolation predictions
            ################################
            data=np.concatenate((sc_X.inverse_transform(x_extra),sc_Y.inverse_transform(y_extra),sc_Y.inverse_transform(pred_extra)),axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('extrapolation.xlsx', index=False, header=name)
            

            #################################  EY: The same as above but Standart deviations are also added.
            # Train Data Predictions (including standard deviations)
            data_train = np.concatenate((sc_X.inverse_transform(x_train), sc_Y.inverse_transform(y_train), sc_Y.inverse_transform(pred_train), sc_Y.inverse_transform(pred_train_std)-sc_Y.mean_), axis=1)
            dfw_train = pd.DataFrame(data_train)
            dfw_train.to_excel('Train_data_predictions_with_std.xlsx', index=False, header=name + ['Standard Deviation'])
            
            # Test Data Predictions (including standard deviations)
            data_test = np.concatenate((sc_X.inverse_transform(x_test), sc_Y.inverse_transform(y_test), sc_Y.inverse_transform(pred), sc_Y.inverse_transform(pred_std)-sc_Y.mean_), axis=1)
            dfw_test = pd.DataFrame(data_test)
            dfw_test.to_excel('predictions_with_std.xlsx', index=False, header=name + ['Standard Deviation'])
            
            # Extrapolation Predictions (including standard deviations)
            data_extra = np.concatenate((sc_X.inverse_transform(x_extra), sc_Y.inverse_transform(y_extra), sc_Y.inverse_transform(pred_extra), sc_Y.inverse_transform(pred_extra_std)-sc_Y.mean_), axis=1)
            dfw_extra = pd.DataFrame(data_extra)
            dfw_extra.to_excel('extrapolation_with_std.xlsx', index=False, header=name + ['Standard Deviation'])

            ################################
            ## Shap Part
            ################################            
            # Fits the explainer
            #explainer = shap.Explainer(gpr.predict, x_test)
            # Calculates the SHAP values - It takes some time
            #shap_values = explainer(x_test)
            
            #shap.summary_plot(shap_values,feature_names=name, plot_type='violin')


            #def gpr_predict(X):
             #   return gpr.predict(X).flatten()

            # Fits the explainer
            #explainer = shap.KernelExplainer(gpr_predict, shap.kmeans(x_train,4))
            # Calculates the SHAP values - It takes some time
            #shap_values = explainer.shap_values(x_test, nsamples='auto')
            
            #shap.summary_plot(shap_values, x_test, feature_names=name, plot_type='violin')
           # plt.show()
            # Create a new figure before saving to ensure the correct figure is saved
           # plt.figure()

            # Save the figure to a file (e.g., PNG)
           # plt.savefig('shap_summary_plot.pdf', dpi=600, bbox_inches='tight')
        
      
            ################################
            ## Write rms to txt file
            ################################
            mse_train = np.sum(np.square(sc_Y.inverse_transform(y_train) - sc_Y.inverse_transform(pred_train))) / np.size(y_train)
            rmse_train = np.sqrt(mse_train)
            
            mse = np.sum(np.square(sc_Y.inverse_transform(y_test) - sc_Y.inverse_transform(pred))) / np.size(y_test)
            rmse = np.sqrt(mse)
            
            ms_extra = np.sum(np.square(sc_Y.inverse_transform(y_extra) - sc_Y.inverse_transform(pred_extra))) / np.size(y_extra)
            rms_extra = np.sqrt(ms_extra)
            
            wfile.write("rms_train=%.4f,rms_pred=%.4f, rms_extra=%.4f\n"%(rmse_train, rmse, rms_extra))
            print(rmse_train, rmse,rms_extra)
                        
            wfile.close()
            os.chdir(cwd)
            

R^2 Score: 0.9995548072365624
0.895660149078173 1.070580389922892 nan


### Result Explaanation
1. "R^2 Score: 0.9995548072360587" present the correlation between the predicted dependent variable and actual data. It is close to 1, which is a great result,  explaining almost all the variance of the dependent variable.
2. "0.021046297976098784 0.025156588574804902 nan" are the RMSE:
        RMSE error of training data predictions with respect to experimental data.
        Second one: RMSE error of test data predictions with respect to experimental data.
        The last one: We do not use this. It is trying to reach information in the "NEW_EXP_DATA_FOR_EXTRAPOLATION.xlsx", but in this file there is no experimental data and gives nan.


# Change Kernel Function

## Kernel Function: White + Periodic

In [1]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sklearn
print(sklearn.__version__)

1.5.0


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, PeriodicKernel

import numpy as np
from sklearn.gaussian_process.kernels import Kernel


# Assuming x_train, y_train, x_test, y_test, x_extra, y_extra, sc_X, sc_Y are defined earlier in your code.

cwd = os.get(cwd)  # Assuming cwd is defined earlier in your code.

os.chdir(cwd)
analyze_name = "GP_Regression"
if not (os.path.exists("./" + str(analyze_name))):
    os.makedirs("./" + str(analyze_name))
os.chdir("./" + str(analyze_name))
wfile = open("results_" + str(analyze_name) + "_selected.txt", "a")
Scale = [1.0]
Alpha = [1e-09]
Normalize = [False]

for s in Scale:
    for a in Alpha:
        for n in Normalize:
            wfile.write(analyze_name + "\n")
            wfile.write("Scale : {} Alpha: {} Normalize: {} \n".format(str(s), str(a), str(n)))
            wfile.write("----------------------------------\n")

            kernel = 1.0 * PeriodicKernel(periodicity=1.0, length_scale=s, length_scale_bounds=(1e-4, 1e5)) + \
                     WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e1))
            gpr = GaussianProcessRegressor(kernel=kernel, alpha=a, normalize_y=n).fit(x_train, y_train)

            score = gpr.score(x_train, y_train)
            print("R^2 Score:", score)

            pred_train = gpr.predict(x_train)
            pred_train = pred_train.reshape(-1, 1)

            pred = gpr.predict(x_test)
            pred = pred.reshape(-1, 1)

            pred_extra = gpr.predict(x_extra)
            pred_extra = pred_extra.reshape(-1, 1)

            pred_train_mean, pred_train_std = gpr.predict(x_train, return_std=True)
            pred_train_mean = pred_train_mean.reshape(-1, 1)
            pred_train_std = pred_train_std.reshape(-1, 1)

            pred_mean, pred_std = gpr.predict(x_test, return_std=True)
            pred_mean = pred_mean.reshape(-1, 1)
            pred_std = pred_std.reshape(-1, 1)

            pred_extra_mean, pred_extra_std = gpr.predict(x_extra, return_std=True)
            pred_extra_mean = pred_extra_mean.reshape(-1, 1)
            pred_extra_std = pred_extra_std.reshape(-1, 1)

            data = np.concatenate((sc_X.inverse_transform(x_train), sc_Y.inverse_transform(y_train),
                                   sc_Y.inverse_transform(pred_train)), axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('Train_data_predictions.xlsx', index=False, header=name)

            data = np.concatenate((sc_X.inverse_transform(x_test), sc_Y.inverse_transform(y_test),
                                   sc_Y.inverse_transform(pred)), axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('predictions.xlsx', index=False, header=name)

            data = np.concatenate((sc_X.inverse_transform(x_extra), sc_Y.inverse_transform(y_extra),
                                   sc_Y.inverse_transform(pred_extra)), axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('extrapolation.xlsx', index=False, header=name)

            data_train = np.concatenate((sc_X.inverse_transform(x_train), sc_Y.inverse_transform(y_train),
                                         sc_Y.inverse_transform(pred_train),
                                         sc_Y.inverse_transform(pred_train_std) - sc_Y.mean_), axis=1)
            dfw_train = pd.DataFrame(data_train)
            dfw_train.to_excel('Train_data_predictions_with_std.xlsx', index=False, header=name + ['Standard Deviation'])

            data_test = np.concatenate((sc_X.inverse_transform(x_test), sc_Y.inverse_transform(y_test),
                                        sc_Y.inverse_transform(pred),
                                        sc_Y.inverse_transform(pred_std) - sc_Y.mean_), axis=1)
            dfw_test = pd.DataFrame(data_test)
            dfw_test.to_excel('predictions_with_std.xlsx', index=False, header=name + ['Standard Deviation'])

            data_extra = np.concatenate((sc_X.inverse_transform(x_extra), sc_Y.inverse_transform(y_extra),
                                         sc_Y.inverse_transform(pred_extra),
                                         sc_Y.inverse_transform(pred_extra_std) - sc_Y.mean_), axis=1)
            dfw_extra = pd.DataFrame(data_extra)
            dfw_extra.to_excel('extrapolation_with_std.xlsx', index=False, header=name + ['Standard Deviation'])

            mse_train = np.sum(np.square(sc_Y.inverse_transform(y_train) - sc_Y.inverse_transform(pred_train))) / np.size(
                y_train)
            rmse_train = np.sqrt(mse_train)

            mse = np.sum(np.square(sc_Y.inverse_transform(y_test) - sc_Y.inverse_transform(pred))) / np.size(y_test)
            rmse = np.sqrt(mse)

            ms_extra = np.sum(
                np.square(sc_Y.inverse_transform(y_extra) - sc_Y.inverse_transform(pred_extra))) / np.size(y_extra)
            rms_extra = np.sqrt(ms_extra)

            wfile.write("rms_train=%.4f,rms_pred=%.4f, rms_extra=%.4f\n" % (rmse_train, rmse, rms_extra))
            print(rmse_train, rmse, rms_extra)

wfile.close()
os.chdir(cwd)

ImportError: cannot import name 'PeriodicKernel' from 'sklearn.gaussian_process.kernels' (/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py)